In [0]:
%pip install --upgrade --force-reinstall /dbfs/tmp/insurance-0.0.1-py3-none-any.whl


In [0]:
%restart_python

In [0]:
import mlflow
from loguru import logger
from pyspark.sql import SparkSession

from insurance.config import ProjectConfig, Tags
from insurance.models.feature_lookup_model import FeatureLookUpModel

In [0]:
spark = SparkSession.builder.getOrCreate()
tags_dict = {"git_sha": "261989george", "branch": "feature3"}
tags = Tags(**tags_dict)

config = ProjectConfig.from_yaml(config_path="../project_config.yml", env="dev")

In [0]:
# Initialize model
fe_model = FeatureLookUpModel(config=config, tags=tags, spark=spark)

In [0]:
# Create feature table
fe_model.create_feature_table()

In [0]:
# Load data
fe_model.load_data()

In [0]:
# Perform feature engineering
fe_model.feature_engineering()

In [0]:
# Train the model
fe_model.train()

In [0]:
# Train the model
fe_model.register_model()

In [0]:
# Lets run prediction on the last production model
# Load test set from Delta table
spark = SparkSession.builder.getOrCreate()

test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").limit(10)

In [0]:
from pyspark.sql.types import StringType

# Convert Id to string
test_set = test_set.withColumn("Id", test_set["Id"].cast(StringType()))

In [0]:
# Drop feature lookup columns and target
X_test = test_set.drop("age", "bmi", "children", config.target)

In [0]:
fe_model = FeatureLookUpModel(config=config, tags=tags, spark=spark)

In [0]:
# Make predictions
predictions = fe_model.load_latest_model_and_predict(X_test)

In [0]:
# Display predictions
logger.info(predictions)